# 07 - Kaggle - bike share system - Random Forest


For problem formulation refer to **"01 - Kaggle - bike share system - problem formulation.ipynb"**.
In section **"02 - Kaggle - bike share system - Data preprocessing.ipynb"** we transformed the raw data and extracted time, date, and dummy matrices. The results are stored in two formats:
 * In `train_prep_orig.csv` and `test_prep_orig.csv` the categorical data are in the original form.
 * In `train_prep_dum.csv` and `test_prep_dum.csv` the categorical data are converted to dummy matrices. 

In section **03 - Kaggle - bike share system - data visualization.ipynb** we ploted the average of customers at different time periods over 2011 and 2012 and discused the pattern of customer behavior. We concluded by a decision to consider these average values as new features of the problem so that the machine learning model will be able to use them as the basis values and the other features apply the necessary correction to make use closer to the actual values. In **04 - Kaggle - bike share system - Adding customer average to the features.ipynb** we added the average customers as new featres to the data sets. We finally, update the data sets as:
 * In `train_prep_orig_avg.csv` and `test_prep_orig_avg.csv` the categorical data are in the original form.
 * In `train_prep_dum_avg.csv` and `test_prep_dum_avg.csv` the categorical data are converted to dummy matrices. 
 
 
In section **5 - Kaggle - bike share system - data analysis.ipynb** we visulaized and analyzed the data. We saw that instead of using average of customers as features, it is better if we use the target to be the deviation of customer from the average value.  

In section **06 - Kaggle - bike share system - Ordinary Linear Regression** we applied an ordinary linear regression. Fitting to deviation data has less `rmsle` than the actual data. Since the system is nonlinear, the linear models behave not so good, and we did not bother getting into the details of model improvement.

In this section, we use random forest regressors an a nonlinear model.


We use the data in `train_prep_orig_avg.csv`, which includes:
<center> temp|  atemp | humidity | windspeed | year | season | month | weekday | hour | workingday | holiday | weather | avg_casual | avg_registered | avg_tot | casual | registered | count
 </center>

- **temp**: temperature in Celsius. 

- **atemp**: "feels like" temperature in Celsius.

- **humidity**: relative humidity

- **windspeed**: wind speed

- **year**: 2011 or 2012

- **season**: Kaggle's [website](https://www.kaggle.com/c/bike-sharing-demand/data) says "`1 = spring, 2 = summer, 3 = fall, 4 = winter`", but the season indecies in the dataset correspond to 
    - 1 = Winter (January-March)
    - 2 = Spring (April-June)
    - 3 = Summer (July-September)
    - 4 = Fall (October-December)
    
- **month**: The month as January=1, December=12  

- **weekday**: The day of the week with Monday=0, Sunday=6

- **hour**: The hours of the datetime 0 - 23

- **workingday**: whether the day is neither a weekend nor holiday
     - 0 = day is weekend or holiday
     - 1 = otherwise 

- **holiday**: whether the day is considered a holiday 
    - 0 = non-holiday
    - 1 = holiday

- **weather**: encoded to make explicit various extreme weather events
    - 1 = Clear, Few clouds, Partly cloudy, Partly cloudy 
    - 2 = Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
    - 3 = Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds)
    - 4 = Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog

- **casual**: number of non-registered user rentals initiated

- **registered**: number of registered user rentals initiated

- **tot**: number of total rentals (casual + registered)

- **avg_casual**: the average number of casual customers per weekday per month per year in a time period
- **avg_registered**:   the average number of registered customers per weekday per month per year in a time period
- **avg_tot**:  the average number of total customers per weekday per month per year in a time period





### Basic settings and importing the libraries

In [18]:
# Resets the namespace by removing all names defined by the user without asking for confirmation
%reset -f


# Panas is used for DataFrame
import pandas as pd

# NumPy is used for manipulating arrays
import numpy as np

# MatPlotLib is used for plotting
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

# the output of plotting commands is displayed inline directly below the code cell that produced it.
%matplotlib inline

# Seaborn is used for statistical plotting
import seaborn as sns

# Used for display dataframes as html tables
from IPython.display import display

The Root Mean Squared Logarithmic Error (RMSLE) is calculated as

$$\epsilon = \sqrt{{1\over n} \sum_{i=1}^n \left[ \log(p_i+1) - \log(a_i+1)\right]^2}$$

*Where:*

- $n_i$ is the number of hours in the test set
- $p_i$ is your predicted count
- $a_i$ is the actual count
- $\log(x)$ is the natural logarithm


First we define a `Score function` and them make a scorer from a performance metric or loss function for use in [GridSearchCV](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and [cross_val_score](http://scikit-learn.org/stable/modules/generated/sklearn.cross_validation.cross_val_score.html). For this purpose we use [sklearn.metrics.make_scorer](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html)

In [19]:
from sklearn.metrics import make_scorer

def rmsle (y, y_pred):
    return np.sqrt(((np.log1p(y_pred) - np.log1p(y))**2).mean())

rmsle_score = make_scorer(rmsle, greater_is_better=False)

### Importing the train data from `train_prep_orig.csv`

In [20]:
#Load train data
data_train = pd.read_csv('data/train_prep_orig_avg.csv')

print "The shape of the train dataset:", data_train.shape
display(data_train.head())


The shape of the train dataset: (10886, 18)


,temp,atemp,humidity,windspeed,year,season,month,weekday,hour,workingday,holiday,weather,avg_casual,avg_registered,avg_tot,casual,registered,tot
0,9.84,14.395,81,0,2011,1,1,5,0,0,0,1,3,21.166667,24.166667,3,13,16
1,9.02,13.635,80,0,2011,1,1,5,1,0,0,1,3,21.166667,24.166667,8,32,40
2,9.02,13.635,80,0,2011,1,1,5,2,0,0,1,1,7.500000,8.500000,5,27,32
3,9.84,14.395,75,0,2011,1,1,5,3,0,0,1,1,7.500000,8.500000,3,10,13
4,9.84,14.395,75,0,2011,1,1,5,4,0,0,1,1,7.500000,8.500000,0,1,1


## Features and targets

In [21]:
cat_var = ['year','season', 'month', 'weekday', 'hour', 'workingday', 'holiday', 'weather']
num_var = ['temp', 'atemp', 'humidity', 'windspeed']
avg_var = ['avg_casual','avg_registered','avg_tot']
dev_var = ['dev_casual','dev_registered','dev_tot']
target_var = ['casual', 'registered', 'tot']


## Deviations of customers from average

In [22]:
data_train['dev_casual'] = data_train['casual'] - data_train['avg_casual'] 
data_train['dev_registered'] = data_train['registered'] - data_train['avg_registered'] 
data_train['dev_tot'] = data_train['tot'] - data_train['avg_tot'] 

data_train.head()

,temp,atemp,humidity,windspeed,year,season,month,weekday,hour,workingday,...,weather,avg_casual,avg_registered,avg_tot,casual,registered,tot,dev_casual,dev_registered,dev_tot
0,9.84,14.395,81,0,2011,1,1,5,0,0,...,1,3,21.166667,24.166667,3,13,16,0,-8.166667,-8.166667
1,9.02,13.635,80,0,2011,1,1,5,1,0,...,1,3,21.166667,24.166667,8,32,40,5,10.833333,15.833333
2,9.02,13.635,80,0,2011,1,1,5,2,0,...,1,1,7.500000,8.500000,5,27,32,4,19.500000,23.500000
3,9.84,14.395,75,0,2011,1,1,5,3,0,...,1,1,7.500000,8.500000,3,10,13,2,2.500000,4.500000
4,9.84,14.395,75,0,2011,1,1,5,4,0,...,1,1,7.500000,8.500000,0,1,1,-1,-6.500000,-7.500000


### Selecting feature ad target variables

In [23]:
X = data_train.loc[:,'temp':'weather']
print "features"
display(X.head())

Y_avg = data_train.loc[:,'avg_casual':'avg_tot']
print "target (average)"
display(Y_avg.head())

Y_org = data_train.loc[:,'casual':'tot']
print "target"
display(Y_org.head())

Y_dev = data_train.loc[:,'dev_casual':'dev_tot']
print "target (deviation)"
display(Y_dev.head())

features


,temp,atemp,humidity,windspeed,year,season,month,weekday,hour,workingday,holiday,weather
0,9.84,14.395,81,0,2011,1,1,5,0,0,0,1
1,9.02,13.635,80,0,2011,1,1,5,1,0,0,1
2,9.02,13.635,80,0,2011,1,1,5,2,0,0,1
3,9.84,14.395,75,0,2011,1,1,5,3,0,0,1
4,9.84,14.395,75,0,2011,1,1,5,4,0,0,1


target (average)


,avg_casual,avg_registered,avg_tot
0,3,21.166667,24.166667
1,3,21.166667,24.166667
2,1,7.500000,8.500000
3,1,7.500000,8.500000
4,1,7.500000,8.500000


target


,casual,registered,tot
0,3,13,16
1,8,32,40
2,5,27,32
3,3,10,13
4,0,1,1


target (deviation)


,dev_casual,dev_registered,dev_tot
0,0,-8.166667,-8.166667
1,5,10.833333,15.833333
2,4,19.500000,23.500000
3,2,2.500000,4.500000
4,-1,-6.500000,-7.500000


### Split to train and test samples

In [24]:
from sklearn.cross_validation import train_test_split

X_train, X_test, Y_avg_train, Y_avg_test, Y_org_train, Y_org_test, Y_dev_train, Y_dev_test = train_test_split(
    X, 
    Y_avg,
    Y_org, 
    Y_dev, 
    test_size = 0.30, 
    random_state = 42)

## MODEL: RANDOM FOREST REGRESSOR

Finding the baseline: here we simply go with the default setting to have a sense of possible outcomes

At frist, we only focus on total number of customers. We may extend the analysis later to casual and registered customers.

We set `random_state=10` for the sake of comparision, otherwise, in each run we get different resuts due to the random nature of the regressor.

To analyze the deviation of customer number from average, we use model's `.score` to calculate $R^2$ for deviation variables and regression score function `sklearn.metrics.r2_score(y_true, y_pred)` to analyze the goodness of fit after we include the average values.

In [41]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

def fit_rf (X_train, Y_train, X_test, Y_test, n_trees = 10):
    rf = RandomForestRegressor(random_state=10, n_estimators= n_trees, criterion='mse')
    rf.fit(X_train,Y_train)    
    print ("Train R^2: %.3f " % rf.score(X_train,Y_train))
    print ("Test R^2: %.3f " % rf.score(X_test,Y_test))
    print ("Train rmsle: %.3f " % rmsle(Y_train, rf.predict(X_train)))
    print ("Test rmsle: %.3f " % rmsle(Y_test, rf.predict(X_test)))
     

def fit_rf_dev (X_train, Y_train, Y_0_train, X_test, Y_test, Y_0_test, n_trees = 10):
    rf = RandomForestRegressor(random_state=10, n_estimators= n_trees, criterion='mse')
    rf.fit(X_train,Y_train)    
    print ("dev Train R^2: %.3f " % rf.score(X_train,Y_train))
    print ("dev Test R^2: %.3f " % rf.score(X_test,Y_test))
    print ("Train R^2: %.3f " % r2_score(Y_train + Y_0_train, rf.predict(X_train) + Y_0_train))
    print ("Test R^2: %.3f " % r2_score(Y_test + Y_0_test, rf.predict(X_test) + Y_0_test)) 
    print ("Train rmsle: %.3f " % rmsle(Y_train + Y_0_train, rf.predict(X_train) + Y_0_train))
    print ("Test rmsle: %.3f " % rmsle(Y_test + Y_0_test, rf.predict(X_test) + Y_0_test))
    
for i in [10, 50, 100, 200, 500]:
    print "\n\n\nNo. trees:", i 
    print "\nFitting the original values (all features)"     
    fit_rf(X_train, Y_org_train.tot, X_test ,Y_org_test.tot,n_trees = i)
    print "\nFitting the deviation values (all features)" 
    fit_rf_dev(X_train = X_train, 
               Y_train = Y_dev_train.dev_tot, 
               Y_0_train = Y_avg_train.avg_tot, 
               X_test = X_test ,
               Y_test = Y_dev_test.dev_tot, 
               Y_0_test = Y_avg_test.avg_tot,
               n_trees = i )




No. trees: 10

Fitting the original values (all features)
Train R^2: 0.989 
Test R^2: 0.944 
Train rmsle: 0.169 
Test rmsle: 0.344 

Fitting the deviation values (all features)
dev Train R^2: 0.973 
dev Test R^2: 0.838 
Train R^2: 0.990 
Test R^2: 0.943 
Train rmsle: 0.187 
Test rmsle: 0.393 



No. trees: 50

Fitting the original values (all features)
Train R^2: 0.992 
Test R^2: 0.951 
Train rmsle: 0.153 
Test rmsle: 0.331 

Fitting the deviation values (all features)
dev Train R^2: 0.979 
dev Test R^2: 0.850 
Train R^2: 0.992 
Test R^2: 0.947 
Train rmsle: 0.160 
Test rmsle: 0.369 



No. trees: 100

Fitting the original values (all features)
Train R^2: 0.993 
Test R^2: 0.952 
Train rmsle: 0.152 
Test rmsle: 0.331 

Fitting the deviation values (all features)
dev Train R^2: 0.980 
dev Test R^2: 0.851 
Train R^2: 0.992 
Test R^2: 0.947 
Train rmsle: 0.161 
Test rmsle: 0.365 



No. trees: 200

Fitting the original values (all features)
Train R^2: 0.993 
Test R^2: 0.952 
Train rmsle

#### note:

So RF shows a much better performance than OLR since it can catch the nonlinrearities. The $R^2$ for train dataset is always more than that of test data set, which suggests we have overfitting. Contrary to linear regression, here `rmsle` of the original values behaves better than the deviation values. 


We see that fitting the deviation values gives less rmsle. In section **5 - Kaggle - bike share system - data analysis.ipynb** we saw that that deviation vales have very low (zero) correlation with `year`, `season`, `month`, `hour`, and `weekday`. So, let's take these out of the data and so the analysis again:

In [43]:
display(X_train.head())

correlated_features = ['temp','atemp','humidity','windspeed','weather','workingday','holiday']

for i in [10, 50, 100, 200, 500]:
    print "\n\n\nNo. trees:", i 
    print "\nFitting the deviation values (all features)" 
    fit_rf_dev(X_train = X_train, 
               Y_train = Y_dev_train.dev_tot, 
               Y_0_train = Y_avg_train.avg_tot, 
               X_test = X_test ,
               Y_test = Y_dev_test.dev_tot, 
               Y_0_test = Y_avg_test.avg_tot,
               n_trees = i )
    print "\nFitting the deviation values (some features)" 
    fit_rf_dev(X_train = X_train[correlated_features],
               Y_train = Y_dev_train.dev_tot, 
               Y_0_train = Y_avg_train.avg_tot, 
               X_test = X_test [correlated_features],
               Y_test = Y_dev_test.dev_tot, 
               Y_0_test = Y_avg_test.avg_tot,
               n_trees = i )


,temp,atemp,humidity,windspeed,year,season,month,weekday,hour,workingday,holiday,weather
613,9.02,9.090,32,39.0007,2011,1,2,1,17,1,0,1
4030,22.14,25.760,68,12.9980,2011,3,9,6,23,0,0,1
3582,26.24,28.790,83,0.0000,2011,3,8,4,4,1,0,1
10101,9.02,11.365,69,8.9981,2012,4,11,1,6,1,0,1
1430,13.12,14.395,81,30.0026,2011,2,4,1,11,1,0,3





No. trees: 10

Fitting the deviation values (all features)
dev Train R^2: 0.973 
dev Test R^2: 0.838 
Train R^2: 0.990 
Test R^2: 0.943 
Train rmsle: 0.187 
Test rmsle: 0.393 

Fitting the deviation values (some features)
dev Train R^2: 0.669 
dev Test R^2: -0.217 
Train R^2: 0.876 
Test R^2: 0.568 
Train rmsle: 0.595 
Test rmsle: 0.816 



No. trees: 50

Fitting the deviation values (all features)
dev Train R^2: 0.979 
dev Test R^2: 0.850 
Train R^2: 0.992 
Test R^2: 0.947 
Train rmsle: 0.160 
Test rmsle: 0.369 

Fitting the deviation values (some features)
dev Train R^2: 0.699 
dev Test R^2: -0.138 
Train R^2: 0.888 
Test R^2: 0.596 
Train rmsle: 0.574 
Test rmsle: 0.798 



No. trees: 100

Fitting the deviation values (all features)
dev Train R^2: 0.980 
dev Test R^2: 0.851 
Train R^2: 0.992 
Test R^2: 0.947 
Train rmsle: 0.161 
Test rmsle: 0.365 

Fitting the deviation values (some features)
dev Train R^2: 0.705 
dev Test R^2: -0.132 
Train R^2: 0.890 
Test R^2: 0.598 
Train rms

### Note:
This shows that for the purpose of our analysis where scoring is based on `rmsle`, contrary to the linear regression, random forest gives poor performance for deviation.